In [2]:
from google.cloud import bigquery
from google.cloud import bigquery_datatransfer
from google.api_core.exceptions import Forbidden, BadRequest, NotFound
import logging
import os
import re 
import json
from datetime import datetime
import pandas as pd 
import sqlite3

In [3]:
def list_datasets(client: bigquery.Client) -> list:
    data_summary = []  # Initialize the list to store views information
    
    try:
        datasets = client.list_datasets()

        if datasets:
            for dataset in datasets:
                dataset_id = dataset.dataset_id
                dataset_ref = client.dataset(dataset_id)

                tables = client.list_tables(dataset_ref)
                for table in tables:
                    table_info = {
                        "dataset_id": dataset_id,
                        "table_id": table.table_id,
                        "type": table.table_type
                    }
                    data_summary.append(table_info)
        else:
            logging.info(f"No datasets found in project {client.project}.")
        
    except Exception as e:
        logging.error(f"An error occurred: {e}")
    
    return data_summary

# Example usage
project_name = "python-dash-game-sales-pd"
client = bigquery.Client(project=project_name)
sources = list_datasets(client)

# Print the sources for verification
print(sources)

c:\Users\GPGHC\AppData\Local\Programs\Python\Python313\Lib\site-packages\google\auth\_default.py:76: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. See the following page for troubleshooting: https://cloud.google.com/docs/authentication/adc-troubleshooting/user-creds. 
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)


[]


In [6]:
conn = sqlite3.connect("data/games_data.db")
try:
    def get_data(_conn):
        # ========== Tratamento inicial do df ============ #
        df = pd.read_sql(f"SELECT * FROM sales_game_data", _conn)
        return df 
    df = get_data(conn)
finally:
    conn.close()
df

DatabaseError: Execution failed on sql 'SELECT * FROM sales_game_data': no such table: sales_game_data

In [6]:
!gcloud auth login

Your browser has been opened to visit:

    https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=32555940559.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A8085%2F&scope=openid+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fuserinfo.email+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcloud-platform+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fappengine.admin+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fsqlservice.login+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcompute+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Faccounts.reauth&state=9cY77qJcK5uZy3BXJpmmm91m1GR1aw&access_type=offline&code_challenge=HQi1YxighPdWCCiMwgMMfmj2jk2l6Gg_-tFR3xDwj9g&code_challenge_method=S256


You are now logged in as [sevengusta@gmail.com].
Your current project is [python-dash-game-sales-pd].  You can change this setting by running:
  $ gcloud config set project PROJECT_ID


In [7]:
# Base directory name
data_atual = datetime.now()
data_formatada = data_atual.strftime("%Y-%m-%d")  # Format: AAAA-MM-DD HH-MM-SS
project_name = data_formatada

# ANSI escape codes for colors
class ColoredFormatter(logging.Formatter):
    RESET = "\033[0m"
    GREEN = "\033[32m"    # Green for INFO
    YELLOW = "\033[33m"   # Yellow for WARNING
    RED = "\033[31m"      # Red for ERROR

    def format(self, record):
        if record.levelno == logging.INFO:
            record.msg = f"{self.GREEN}{record.msg}{self.RESET}"
        elif record.levelno == logging.WARNING:
            record.msg = f"{self.YELLOW}{record.msg}{self.RESET}"
        elif record.levelno == logging.ERROR:
            record.msg = f"{self.RED}{record.msg}{self.RESET}"
        return super().format(record)

# Configure logging
logger = logging.getLogger()
if not logger.hasHandlers():  # Check if handlers are already added
    handler = logging.StreamHandler()
    handler.setFormatter(ColoredFormatter('%(asctime)s - %(levelname)s - %(message)s'))
    logger.addHandler(handler)
    logger.setLevel(logging.INFO)

# Code to list all content inside Datasets and dataTransfers from BigQuery
def list_datasets(client_name):
    data_summary = []  # Initialize the list to store views information
    datasets = client_name.list_datasets()

    if datasets:
        for dataset in datasets:
            dataset_id = dataset.dataset_id
            dataset_ref = client_name.dataset(dataset_id)

            tables = client_name.list_tables(dataset_ref)
            for table in tables:
                table_info = {
                    "dataset_id": dataset_id,
                    "table_id": table.table_id,
                    "type": table.table_type
                }
                data_summary.append(table_info)
        return  data_summary
    else:
        logger.info(f"Nenhum dataset encontrado no projeto {client_name.project}.")
        return []

def list_data_transfers(project_id: str):
    client = bigquery_datatransfer.DataTransferServiceClient()
    parent = f"projects/{project_id}/locations/US"
    transfer_summary = []  # Initialize the list to store transfer information

    for transfer_config in client.list_transfer_configs(parent=parent):
        transfer_info = {
                "name_transfer": transfer_config.display_name,
                "dataset_id": transfer_config.destination_dataset_id,
                "table_id": transfer_config.params.get("destination_table_name_template", "Não especificado"),
                "sql": transfer_config.params.get("query", None),
                "type_transfer": transfer_config.data_source_id.replace("_", " ").title() if transfer_config.destination_dataset_id != 'gateway_t' else 'CSW Data Gateway',
                "schedule": transfer_config.schedule or ("Não especificado" if transfer_config.destination_dataset_id != 'gateway_t' else "On Demand"),
                "url": f"https://console.cloud.google.com/bigquery/transfers/locations/us/configs/{transfer_config.name.split('/')[-1]}/runs?inv=1&invt=Ablxwg&project={project_id}"
            }
        transfer_summary.append(transfer_info)
    return transfer_summary

# Create BigQuery Files
def save_sql(relative_path, resource_id, query):
    file_path = os.path.join(relative_path, f"{resource_id}.sql")
    
    formatted_sql = query.replace('\r\n', '\n').strip()  # Normalize newlines and strip whitespace
    
    with open(file_path, 'w', encoding='utf-8') as sql_file:
        sql_file.write(formatted_sql)
    
    logger.info(f"Formatted SQL saved to {file_path}")

def save_schema(dataset_id, resource_id, table):
    schema_info = [
        {
            "name": field.name,
            "type": field.field_type,
            "mode": field.mode,
            "description": field.description,
        }
        for field in table.schema
    ]
    # Save the schema_info to a JSON file
    file_path = os.path.join(dataset_id, f"{resource_id}.json")
    with open(file_path, 'w', encoding='utf-8') as json_file:
        json.dump(schema_info, json_file, ensure_ascii=False, indent=2)

    logger.info(f"Schema information saved to {file_path}")
    
def process_external_tables(external_table_ids):   
    logger.warning(f"Processando transferências de dados de outros projetos (np)")
    while external_table_ids:
        new_external_table_ids = set()
        

        for external_id in external_table_ids:
            external_project_id = external_id[1]
            base_dir_sql = os.path.join(project_name,"SQLS" , external_project_id)
            base_dir_schema = os.path.join(project_name,"SCHEMAS" ,external_project_id)
            try: 
                full_table_id = external_id[0]
                source = client.get_table(full_table_id)
                os.makedirs(base_dir_sql, exist_ok=True)
                os.makedirs(base_dir_schema, exist_ok=True)
                
                dataset_dir_sql = os.path.join(base_dir_sql, external_id[2])
                dataset_dir_schema = os.path.join(base_dir_schema, external_id[2])
                os.makedirs(dataset_dir_sql, exist_ok=True)
                os.makedirs(dataset_dir_schema, exist_ok=True)
                
                if source.table_type == 'VIEW':
                    view_query = source.view_query
                    
                    for match in re.finditer(compiled_regex, view_query):
                        new_external_table_ids.add((
                            f"{match.group(1)}.{match.group(2)}.{match.group(3)}",
                            match.group(1),  # Projeto
                            match.group(2),  # Dataset
                            match.group(3)   # Tabela
                        ))
                
                    #SQL
                    save_sql(dataset_dir_sql, source.table_id, view_query)
                    
                    #JSON
                    save_schema(dataset_dir_schema, source.table_id, source)
                    
                    
                if source.table_type == 'TABLE':
                    table_query = f"SELECT t.* FROM `{project_id}.{dataset_id}.{table_id}` AS t"
                    
                    #SQL
                    save_sql(dataset_dir_sql, source.table_id, table_query)
                    
                    #JSON
                    save_schema(dataset_dir_schema, source.table_id, source)
                    
                    
            except Forbidden as e:
                logger.error(f'Você não possuí acesso a essa tabela: {e}')
            except BadRequest as e:
                logger.error(f'Você não possuí acesso a essa tabela: {e}')
            
        external_table_ids = new_external_table_ids   

external_dict = {
    "python-dash-game-sales-pd": "python-dash-game-sales",
}

# Main loop to get project IDs from the user
while True:
    external_table_ids = set()
    projects = ["python-dash-game-sales-pd"]
    
    for project_id in projects:
        # Initialize summaries
        data_summary = []
        transfer_summary = {}
    
        logger.warning(f'Nome do projeto Procurado: {project_id}')
            
        try:
            client = bigquery.Client(project=project_id)
            datasets = list(client.list_datasets())  # Convert the iterator to a list
            datasets[0] # Verify if exist any dataset in this project
            
            data_summary = list_datasets(client)
            transfer_summary = list_data_transfers(project_id)
            table_names = [transfer['table_id'] for transfer in transfer_summary] # Store the of each table with transfer
            logger.info('Dados Foram Acessados com sucesso.')
            base_dir_sql = os.path.join(project_name,"SQLS" , project_id)
            base_dir_schema = os.path.join(project_name,"SCHEMAS" ,project_id)
            os.makedirs(base_dir_sql, exist_ok=True)
            os.makedirs(base_dir_schema, exist_ok=True)
            
        except Exception as e:
            logger.error(f'Erro ao tentar acessar o projeto BigQuery: {e}')
            logger.warning('Por favor, selecione um projeto válido.')
        
        
        # Regular expression to match external table references
        regex = rf"({external_dict[project_id]})\.([^`]+)\.([^`]+)"
        compiled_regex = re.compile(regex)

        # Save views and their schemas
        for table in data_summary:
            dataset_id = table['dataset_id']
            table_id = table['table_id']
            full_table_id = f"{project_id}.{table['dataset_id']}.{table['table_id']}"
            dataset_dir_sql = os.path.join(base_dir_sql, dataset_id)
            dataset_dir_schema = os.path.join(base_dir_schema, dataset_id)
            os.makedirs(dataset_dir_sql, exist_ok=True)
            os.makedirs(dataset_dir_schema, exist_ok=True)
            source = client.get_table(full_table_id)
            if table['type'] == "VIEW":
                view_query = source.view_query 
                
                for match in re.finditer(compiled_regex, view_query):
                    # Check if any part of the first group matches "bcs-latam-it-lake"
                    if external_dict[project_id] in match.group(1):
                        external_table_ids.add((
                            f"{match.group(1)}.{match.group(2)}.{match.group(3)}",
                            match.group(1),  # Project
                            match.group(2),  # Dataset
                            match.group(3)   # Table
                        ))
                # SQL
                save_sql(dataset_dir_sql, table_id, view_query)
                
                # JSON
                save_schema(dataset_dir_schema, table_id, source)
                
            elif table['type'] == "TABLE" and table_id not in table_names :
                table_query = f"SELECT t.* FROM `{project_id}.{dataset_id}.{table_id}` AS t"
                # SQL
                save_sql(dataset_dir_sql, table_id, table_query)
                
                # JSON
                save_schema(dataset_dir_schema, table_id, source)
                

        # Save transfer information
        
        for table in transfer_summary:
            
            dataset_id = table['dataset_id']
            table_id = table['table_id']
            full_table_id = f"{project_id}.{table['dataset_id']}.{table['table_id']}"
            dataset_dir_sql = os.path.join(base_dir_sql, dataset_id)
            dataset_dir_schema = os.path.join(base_dir_schema, dataset_id)
            os.makedirs(dataset_dir_sql, exist_ok=True)
            os.makedirs(dataset_dir_schema, exist_ok=True)
            source = client.get_table(full_table_id)
            if table['sql']:
                
                # SQL
                for match in re.finditer(compiled_regex, table['sql']):
                    # Check if any part of the first group matches "bcs-latam-it-lake"
                    if  match.group(1) in external_dict[project_id]:
                        external_table_ids.add((
                            f"{match.group(1)}.{match.group(2)}.{match.group(3)}",
                            match.group(1),  # Project
                            match.group(2),  # Dataset
                            match.group(3)   # Table
                        ))
                        
                        
                save_sql(dataset_dir_sql, table_id, table['sql'])
                    
                # JSON
                save_schema(dataset_dir_schema, table_id, source)
            else:
                table_query = f"SELECT t.* FROM `{project_id}.{dataset_id}.{table_id}` AS t"
                save_sql(dataset_dir_sql, table_id, table_query)
                    
                # JSON
                save_schema(dataset_dir_schema, table_id, source)
                
        # processing external tables
        process_external_tables(external_table_ids)
    break

2025-05-11 16:57:55,796 - WARNING - Nome do projeto Procurado: python-dash-game-sales-pd
2025-05-11 16:57:59,819 - ERROR - Erro ao tentar acessar o projeto BigQuery: list index out of range
2025-05-11 16:57:59,820 - WARNING - Por favor, selecione um projeto válido.
2025-05-11 16:57:59,820 - WARNING - Processando transferências de dados de outros projetos (np)
